# NBDT vs Baseline Inference

In [1]:
# import custom modules
import sys
sys.path.insert(0, "../src/util")
sys.path.insert(0, "../src/model")
sys.path.insert(0, "../src/data_util")

# imports for model
import torch
import torchvision
import os
import torch.optim as optim
import numpy as np
import torch.nn as nn
import torch.nn.functional as F
import torch.utils.checkpoint as cp
from torchvision import datasets, models, transforms
from sklearn.metrics import f1_score

from baseline import *

from nbdt.model import SoftNBDT
from nbdt.loss import SoftTreeSupLoss

from wn_utils import *
from graph import *
from dir_grab import *
from hierarchy import *
from debug_data import *
from write_to_json import *
from loss import *

from datetime import datetime

not enough values to unpack (expected 2, got 0)


In [3]:
with open('../config/data-params.json') as fh:
    data_cfg = json.load(fh)
    print('---> loaded data config')

with open('../config/model-params.json') as fh:
    model_cfg = json.load(fh)
    print('---> loaded model config')

---> loaded data config
---> loaded model config


In [14]:
# Detect if we have a GPU available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [16]:
# load model that was already trained
model_weights = torch.load(os.path.join('..',data_cfg['hierarchyModelPath']))
model, input_size = initialize_model(
    model_cfg['modelName'],
    model_cfg['nClasses'],
    feature_extract = 'False'
)
model = model.to(device) # make model use GPU
model.load_state_dict(model_weights)

---> Begin model initialization...
---> Finished model initialization.


<All keys matched successfully>

In [17]:
# create test data loader
data_transforms = {
    'valid_snakes_r1': transforms.Compose([
        transforms.Resize(input_size),
        transforms.CenterCrop(input_size),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ])
}
    
# Create test datasets
image_datasets = {
    'valid_snakes_r1': datasets.ImageFolder(os.path.join('../' + data_cfg['dataDir'], 'valid_snakes_r1'), data_transforms['valid_snakes_r1'])  
}
    
# Create training and validation dataloaders
dataloaders_dict = {
    x: torch.utils.data.DataLoader(
        image_datasets[x],
        batch_size=16,
        shuffle=True,
        num_workers=4
    ) for x in [
        'valid_snakes_r1'
    ]
}

In [28]:
model.eval()

running_corrects = 0
fscore = []

# iterate over data
for inputs, labels in dataloaders_dict['valid_snakes_r1']:
    inputs = inputs.to(device)
    labels = labels.to(device)

    with torch.no_grad():
        outputs = model(inputs)
        _, preds = torch.max(outputs, 1)

    # statistics
    labels_cpu = labels.cpu().numpy()
    predictions_cpu = preds.cpu().numpy()
    Fscore = f1_score(labels_cpu, predictions_cpu, average='macro')
    fscore.append(Fscore)
    running_corrects += torch.sum(preds == labels.data)

In [29]:
epoch_acc = running_corrects.double() / len(dataloaders_dict['valid_snakes_r1'].dataset)
epoch_fscore = np.average(np.array(fscore))

print('{} Acc: {:.4f} F: {:.3f}'.format('CNN Test', epoch_acc, epoch_fscore))

CNN Test Acc: 0.6617 F: 0.516


In [36]:
nbdt_model = SoftNBDT(
    model = model,
    dataset = 'snakes', 
    hierarchy='induced-densenet121',
    path_graph = "../data/hierarchies/snakes/graph-induced-densenet121.json",
    path_wnids = "../data/wnids/snakes.txt"
)

In [37]:
nbdt_model.eval()

nbdt_running_corrects = 0
nbdt_fscore = []

# iterate over data
for inputs, labels in dataloaders_dict['valid_snakes_r1']:
    inputs = inputs.to(device)
    labels = labels.to(device)

    with torch.no_grad():
        outputs = nbdt_model(inputs)
        _, preds = torch.max(outputs, 1)

    # statistics
    nbdt_labels_cpu = labels.cpu().numpy()
    nbdt_predictions_cpu = preds.cpu().numpy()
    nbdt_Fscore = f1_score(nbdt_labels_cpu, nbdt_predictions_cpu, average='macro')
    nbdt_fscore.append(nbdt_Fscore)
    nbdt_running_corrects += torch.sum(preds == labels.data)

In [38]:
nbdt_epoch_acc = nbdt_running_corrects.double() / len(dataloaders_dict['valid_snakes_r1'].dataset)
nbdt_epoch_fscore = np.average(np.array(nbdt_fscore))

print('{} Acc: {:.4f} F: {:.3f}'.format('NBDT Test', nbdt_epoch_acc, nbdt_epoch_fscore))

NBDT Test Acc: 0.4450 F: 0.302


In [45]:
import pandas as pd
df = pd.DataFrame([
    [epoch_acc.item(), epoch_fscore],
    [nbdt_epoch_acc.item(), nbdt_epoch_fscore]
])
df.columns = ['acc', 'fscore']
df.index = ['baseline_cnn', 'nbdt']
df

,acc,fscore
baseline_cnn,0.661744,0.516375
nbdt,0.445032,0.301830


In [46]:
df.to_csv('../data/out/nbdt_vs_baseline.csv')